---
title: "変分推論１"
subtitle: "K-平均アルゴリズム"
author: "司馬博文"
date: 2/3/2024
date-modified: 8/12/2024
categories: [Computation, Python]
bibliography: 
    - ../../../assets/2023.bib
    - ../../../assets/2024.bib
    - ../../../assets/2025.bib
csl: ../../../assets/apalike.csl
abstract-title: 概要
abstract: |
    本稿では，[$K$-平均アルゴリズム](https://ja.wikipedia.org/wiki/K%E5%B9%B3%E5%9D%87%E6%B3%95) によるクラスタリングの考え方と問題点を，Python による実演を通じてみる．[次稿](VI2.qmd) で，$K$-平均アルゴリズムの model-aware な一般化として [EM アルゴリズム](https://ja.wikipedia.org/wiki/EM%E3%82%A2%E3%83%AB%E3%82%B4%E3%83%AA%E3%82%BA%E3%83%A0) を説明し，その共通の問題点「初期値依存性」と「局所解へのトラップ」の数理的な理解を目指す．
listing: 
    -   id: vi-listing
        type: grid
        sort: false
        contents:
            - VI2.qmd
            - VI3.qmd
            - ../Kernels/Kernel.qmd
        date-format: iso
        fields: [title,image,date,subtitle]
---

::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\operatorname{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\mathtt{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::





より図が見やすい PDF 版は [こちら](report1.pdf)．

### 関連ページ {.unnumbered .unlisted}

::: {#vi-listing}
:::


## 導入

### 歴史 {#sec-history}

ハード $K$-平均法はモデルフリーのクラスタリングアルゴリズムである．Voronoi 分割による競争学習の一形態とも見れる．^[[@MacKay2003 p.285]．]

一方で原論文 [@Lloyd1982] では，[パルス符号変調](https://ja.wikipedia.org/wiki/%E3%83%91%E3%83%AB%E3%82%B9%E7%AC%A6%E5%8F%B7%E5%A4%89%E8%AA%BF) の文脈で，アナログ信号の量子化の方法として提案している．^[Stuart P. Lloyd は 1957 年にメモランダムとして発表していたが，論文の形で出版されたのが 1982 である．$K$-means という名前の初出は [@MacQueen1967] とされている．]

実際，$K$-平均法は（非可逆）データ圧縮にも用いられる．クラスター中心での画像の値と，それ以外では帰属先のクラスター番号のみを保存すれば良いというのである．このようなアプローチを **ベクトル量子化** (vector quantization) という．^[[@MacKay2003 p.284]，[@Bishop2006 p.429]，[@Murphy2022 p.722] 21.3.3節．クラスター中心は **符号表ベクトル** または 代表ベクトル (code-book vector) という．]

ソフト $K$-平均法とは，このようなデータ点のクラスターへの一意な割り当てを，ソフトマックス関数を用いて軟化したアルゴリズムであり，多少アルゴリズムとしての振る舞いは改善するとされている．

### 最適化アルゴリズムとしての見方

$N$ 個のデータ $\{x_n\}_{n=1}^N$ の $K$ クラスへの $K$-平均クラスタリングアルゴリズムは，ハードとソフトの二種類存在するが，いずれも
$$
J:=\sum_{n=1}^N\sum_{k=1}^Kr_{nk}\norm{x_n-\mu_k}^2
$$
という損失関数の逐次最小化アルゴリズムとみなせる．

この見方は，[EM アルゴリズム](VI2.qmd) への一般化の軸となる．

しかしこの目的関数 $J$ が非凸関数であることが，第 [-@sec-result] 節で示す実験結果で見る通り，アルゴリズムに強い初期値依存性をもたらす．

### $K$-平均法の変種

#### $K$-means++

$K$-平均法アルゴリズムの初期値依存性を，クラスタがより互いに離れるようにクラス割り当て法を修正することで対策した **最遠点クラスタリング** [@Gonzalez1985] が提案された．これは **$K$-means++** [@David-Sergei2007] とも呼ばれており，データ圧縮法として用いた場合は復元誤差が $O(\log K)$ でバウンド出来ることも示されている．

#### $K$-medoids

[medoid](https://en.wikipedia.org/wiki/Medoid) とは，データの間に何らかの「類似度」が定義されている際に，クラスター内で他データ点との類似度が最大になる点を意味する．

この medoid を mean の代わりに用いたアルゴリズムは，データ内の外れ値により強くなるとされる．

$K$-medoids クラスタリングを行うアルゴリズムに Partitioning Around Medoids (PAM) [@Kaufmann-Rousseeuw1987] や Voronoi iteration [@Park-Jun2009] が知られている．

### 用いるデータ {#sec-data}

実際のコードとデータを用いて $K$-平均法を解説する．

まずは，解説にために作られた，次のような３つのクラスタからなる２次元のデータを考え，これの正しいクラスタリングを目指す．


In [ ]:
#| echo: false
import pandas as pd
data1 = pd.read_csv('mixture1.dat', delimiter="\t" , header=None)
data1_2d = data1.iloc[:, 1:3].to_numpy()

In [ ]:
#| echo: false
import matplotlib.pyplot as plt
import numpy as np

markers1 = data1.iloc[:, 0]
x1 = data1.iloc[:, 1]
y1 = data1.iloc[:, 2]

plt.figure(figsize=(3.5, 3))
for marker in np.unique(markers1):
    plt.scatter(x1[markers1 == marker], y1[markers1 == marker], label=f'Cluster {marker}', marker=f'${marker}$') 

plt.title('Scatter Plot of the Data')
plt.show()

## ハード $K$-平均法 {#sec-hard-k-means}

### アルゴリズムの説明

head $K$-means algorithm はデータ $\{x^{(n)}\}_{n=1}^N\subset\R^I$ とクラスタ数 $K\in\N^+$，そして初期クラスター中心 $(m^{(k)})_{k=1}^K\in(\R^I)^K$ の３組をパラメータに持つ．

soft $K$-means algorithm [-@sec-soft-k-means] はさらに硬度パラメータ $\beta\in\R_+$ を持つ．

`numpy` の提供する行列積を利用して，これを Python により実装した例を以下に示す．ソフト $K$-平均法の実装と対比できるように，負担率を通じた実装を意識した例である．

アノテーションを付してあるので，該当箇所（右端の丸囲み数字）をクリックすることで適宜解説が読めるようになっている．


In [ ]:
#| echo: false
import math
from itertools import permutations

def d(x,y):
    """
    ２次元での Euclid 距離を計算する関数

    Parameter:
    - x,y: (2,)-numpy.ndarray
    """
    return math.sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2)

def normalize(x):
    """
    ラベル番号データを昇順にする関数

    Parameter:
    - x: (N,)-numpy.ndarray
    """
    labels = [x[0]]
    for i in x:
        if i not in labels:
            labels.append(i)

    conditions = [x == labels[i] for i in range(len(labels))]
    choices = [i + 1 for i in range(len(labels))]

    return np.select(conditions, choices)

def normalize_abnormal(x):
    """
    ラベル番号データを昇順にする関数

    Parameter:
    - x: (N,)-numpy.ndarray
    """
    labels = [x[0]]
    for i in x:
        if i not in labels:
            labels.append(i)

    conditions = [x == labels[i] for i in range(len(labels))]
    choices = [1,3,2]

    return np.select(conditions, choices)

def accuracy(ans, pred):
    """
    クラスタリング結果と正解値とを比べ，正答数と正解率を返す関数

    Parameters:
    - ans: (N,)-numpy.ndarray 正解値
    - pred: (N,)-numpy.ndarray 予測値

    ただし，成分は 1,2,3 のみとする．
    """
    perms = permutations([1, 2, 3])
    num_correct = []

    for perm in perms:
        mapping = {1: perm[0], 2: perm[1], 3: perm[2]}
        permuted_pred = np.array([mapping[value] for value in pred])
        num_correct.append(np.sum(ans == permuted_pred))

    total_correct = max(num_correct)
    accuracy = total_correct / len(ans)
    return total_correct, accuracy

def accuracy4(ans, pred):
    """
    正解値と予測値を比べ，正答数と正解率を print する関数

    Parameters:
    - ans: (N,)-numpy.ndarray 正解値
    - pred: (N,)-numpy.ndarray 予測値
    """
    perms = permutations([1, 2, 3, 4])
    num_correct = []

    for perm in perms:
        mapping = {1: perm[0], 2: perm[1], 3: perm[2], 4: perm[3]}
        permuted_pred = np.array([mapping[value] for value in pred])
        num_correct.append(np.sum(ans == permuted_pred))

    total_correct = max(num_correct)
    accuracy = total_correct / len(ans)
    return total_correct, accuracy

```{.python}
def hkmeans_2d(data, K, init, max_iter=100):
    """
    ２次元データに対するハード K-平均法の実装例．

    Parameters:
    - data: (N,2)-numpy.ndarray
    - K: int クラスター数
    - init: (2,K)-numpy.ndarray 初期値

    Returns:
    - clusters: (N,)-numpy.ndarray クラスター番号
    """

    N = data.shape[0]  # <1>
    I = data.shape[1]  # <2>
    m = init  # <3>
    r = np.zeros((K, N), dtype=float)  # <4>

    for _ in range(max_iter):
        # Assignment Step
        for i in range(N):
            distances = np.array([d(data[i], m[:,k]) for k in range(K)]) # <5>
            k_hat = np.argmin(distances)  # <6>
            r[:,i] = 0  # <7>
            r[k_hat,i] = 1
        
        # Update Step
        new_m = np.zeros_like(m, dtype=float) # <8>
        numerator = np.dot(r, data)  # <9>
        denominator = np.sum(r, axis=1) # <10>
        for k in range(K): # <11>
            if denominator[k] > 0:
                new_m[:,k] = numerator[k] / denominator[k]
            else:
                new_m[:,k] = m[:,k]

        if np.allclose(m, new_m): # <12>
            break
        m = new_m
    
    return np.argmax(r, axis=0)  # <13>
```

1. データ数を取得している．
2. データの次元を取得している．今回はすべて２次元データを用いる．
3. クラスター中心に引数として受け取った初期値を代入. $2×K$-行列であることに注意．
4. 負担率を $K×N$-行列として格納している．その理由は後ほど行列積を通じた計算を行うためである．`dtype=float` の理由は後述．
5. この `distances` 変数は `(K,)-numpy.ndarray` になる．すなわち，第 $k$ 成分が，第 $k$ クラスター中心との距離となっているようなベクトルである．ただし，`d` は Euclid 距離を計算する関数として定義済みとした．
6. 距離が最小となるクラスター番号 $\hat{k}:=[\argmin_{k\in[K]}d(m_k,x_i)]$ を，$i\in[N]$ 番目のデータについて求める．
7. $\hat{k}$ に基づいて負担率を更新するが，ループ内で前回の結果をリセットする必要があることに注意．
8. ここで `dtype=float` と指定しないと，初め引数 `init` が整数のみで構成されていた場合に，Python の自動型付機能が `int` 型だと判定し，クラスター中心 `m` の値が整数に限られてしまう．すると，アルゴリズムがすぐに手頃な格子点に収束してしまう．
9. `numpy` の行列積を計算する関数 `np.dot` を使用している．更新式
$$
m^{(k)}\gets\frac{\sum_{n=1}^Nr^{(n)}_kx^{(n)}}{\sum_{n=1}^Nr^{(n)}_k}
$$
の分子を行列積と見たのである．
10. 分母 (denominator) は $(K,N)$-行列 `r` の行和として得られる．
11. ゼロによる除算が起こらないように場合わけをしている．
12. クラスター中心がもはや変わらない場合はアルゴリズムを終了する．
13. 負担率の最も大きいクラスター番号を返す．今回は `hat_k` の列をそのまま返せば良いが，soft $K$-means アルゴリズムにも通じる形で実装した．

::: {.callout-caution icon="false" title="注：実際に用いる実装" collapse="true"}

ただし，本記事の背後では次の実装を用いる．

クラスター中心の推移のヒストリーを保存して図示に利用したり，負担率 `r` の中身を見たりすることが出来るようにするため，assignment step と update step とに分けてクラスメソッドとして実装し，`run` メソッドでそれらを呼び出すようにしている．これに `fetch_cluster` と `fetch_history` メソッドを加えることで，クラスター番号とクラスター中心の推移を取得することが出来る．フィールド `.r` から（最終的な）負担率を見ることもできる．


In [ ]:
#| code-fold: true
class kmeans_2d:
    """
    ２次元データに対するソフト K-平均法の実装．

    Usage:
        kmeans = kmeans_2d(data, K, init, beta)
        kmeans.run()

    Parameters:
    - data: (N,2)-numpy.ndarray
    - K: int クラスター数
    - init: (2,K)-numpy.ndarray 初期値
    - beta: float 硬度パラメータ
    """

    def __init__(self, data, K, init, beta, max_iter=100):
        self.data = np.array(data, dtype=float)
        self.K = K
        self.init = np.array(init, dtype=float)
        self.beta = float(beta)
        self.max_iter = max_iter
        self.N = data.shape[0]  # データ数
        self.I = data.shape[1]  # 次元数 今回は２
        self.m = init  # クラスター中心の初期化．2×K行列．
        self.r = np.zeros((K, self.N), dtype=float)  # 負担率．K×N行列．
        self.history = [init.copy()] # クラスター中心の履歴．2×K行列．
    
    def soft_assigment(self):
        """soft K-means の場合の負担率の更新"""
        for i in range(self.N):
            distances = np.array([d(self.data[i], self.m[:,j]) ** 2 for j in range(self.K)]) # (N,)-numpy.ndarray
            denominator_ = np.sum(np.exp(-self.beta * distances))  # 分母
            self.r[:,i] = np.exp(- self.beta * distances) / denominator_

    def hard_assigment(self):
        """hard K-means の場合の負担率の更新"""
        for i in range(self.N):
            distances = np.array([d(self.data[i], self.m[:,j]) for j in range(self.K)]) # (N,)-numpy.ndarray
            k_hat = np.argmin(distances)  # 最小距離のクラスター番号
            self.r[:,i] = 0  # 前のループの結果をリセット
            self.r[k_hat,i] = 1
    
    def update(self):
        """クラスター中心の更新"""
        new_m = np.zeros_like(self.m, dtype=float) # ここで float にしないと，クラスター中心が整数に限られてしまう．
        numerator = np.dot(self.r, self.data)  # (K,2)-numpy.ndarray
        denominator = np.sum(self.r, axis=1)  # 各クラスターの負担率の和
        for k in range(self.K):
            if denominator[k] > 0:
                new_m[:,k] = numerator[k] / denominator[k]
            else:
                new_m[:,k] = self.m[:,k]
        self.m = new_m

    def fetch_cluster(self):
        """最終的なクラスター番号を格納した (N,)-array を返す"""
        return np.argmax(self.r, axis=0)
    
    def fetch_history(self):
        """クラスター中心の履歴を格納したリストを，３次元の np.array に変換して返す"""
        return np.stack(self.history, axis=0)

    def run_soft(self):
        """soft K-means アルゴリズムの実行"""
        for _ in range(self.max_iter):
            self.soft_assigment()
            self.update()
            self.history.append(self.m.copy())
            if np.allclose(self.history[-1], self.history[-2]):
                break
    
    def run_hard(self):
        """hard K-means アルゴリズムの実行"""
        for _ in range(self.max_iter):
            self.hard_assigment()
            self.update()
            self.history.append(self.m.copy())
            if np.allclose(self.history[-1], self.history[-2]):
                break

なお，この実装は $\beta\ge500$ などの場合にオーバーフローが起こることに注意．これへの対処は `logsumexp` の使用などが考えられる．

:::

### 初期値依存性

次の２つの初期値を与えてみる．
$$
m_1:=\vctr{4}{0},\quad m_2:=\vctr{1}{4},\quad m_3=\vctr{-1}{1},
$$
と，$m_2,m_3$ は変えずに $m_1$ の $y$-座標を $1$ だけ下げたもの
$$
m_1':=\vctr{4}{-1}
$$
とを初期値として与えてみる．


In [ ]:
#| echo: false
#| label: fig-1
#| fig-cap: ハード K-平均法によるクラスタリングの結果．初期値は $(m_1,m_2,m_3)=\paren{\vctr{4}{0},\vctr{1}{4},\vctr{-1}{1}}$．赤丸で囲まれている点がクラスター中心 (CoC / Center of Cluster) の初期値で，その後の移動が図示されている．
initial_points = np.array([[-1,1],[1,4],[4,0]])

result = kmeans_2d(data1_2d, 3, initial_points.T, 0.1)
result.run_hard()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [2,1,0]
default = [0,1,2]
markerstyle = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markerstyle[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Hard K-means')

axs[0].legend()

# markerを使用したプロット
for marker in np.unique(markers1):
    axs[1].scatter(x1[markers1 == marker], y1[markers1 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

axs[1].set_title('Answer')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct, accr = accuracy(ans, normalize(pred))
iteration = history.shape[0]
print(f'正解数: {num_correct}     正解率: {accr*100:.1f} %     反復数: {iteration} 回')

別の初期値を与えてみる（右下の点 $m_1$ を $1$ だけ下に下げただけ）：
$$
\vctr{4}{0}=m_1\mapsto m_1':=\vctr{4}{-1}
$$


In [ ]:
#| echo: false
#| label: fig-2
#| fig-cap: ハード K-平均法によるクラスタリングの結果．初期値は $(m_1',m_2,m_3)=\paren{\vctr{4}{-1},\vctr{1}{4},\vctr{-1}{1}}$．

initial_points = np.array([[-1,1],[1,4],[4,-1]])

result = kmeans_2d(data1_2d, 3, initial_points.T, 0.1)
result.run_hard()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markerstyle = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markerstyle[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Hard K-means')

axs[0].legend()

# markerを使用したプロット
for marker in np.unique(markers1):
    axs[1].scatter(x1[markers1 == marker], y1[markers1 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

axs[1].set_title('Answer')

axs[1].legend()

plt.tight_layout()
plt.show()

center_2 = history[-1, :, :]

num_correct, accr = accuracy(ans, normalize(pred))
iteration = history.shape[0]
print(f'正解数: {num_correct}     正解率: {accr*100:.1f} %     反復数: {iteration} 回')

結果が全く変わり，$(m_1',m_2,m_3)$ を与えた方が，大きく正解に近づいている．具体的には，右下の初期値 $m_1$ は右上の島に行くが，$m_1'$ は左下の島に行ってくれる．

**ハード $K$-平均アルゴリズムは初期値に敏感である** ことがよく分かる．

### 局所解への収束

直前の結果[-@fig-2]ではクラスター２と３の境界線で４つのミスを犯しており，これを修正できないか試したい．

そこで，答えに近いように，
$$
m_1\gets\vctr{2.5}{2},\;\; m_2\gets\vctr{-1}{-1},\;\; m_3\gets\vctr{1}{-2},
$$
を初期値として与えてみて，正答率の変化を観察する．


In [ ]:
#| echo: false
initial_points = np.array([[-1,-1],[1,-2],[2.5,2]])
result = kmeans_2d(data1_2d, 3, initial_points.T, 0.1)
result.run_hard()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [2,0,1]
default = [0,1,2]
markerstyle = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markerstyle[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=1')

axs[0].legend()

# markerを使用したプロット
for marker in np.unique(markers1):
    axs[1].scatter(x1[markers1 == marker], y1[markers1 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

axs[1].set_title('Answer')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct, accr = accuracy(ans, normalize(pred))
iteration = history.shape[0]
print(f'正解数: {num_correct}     正解率: {accr*100:.1f} %     反復数: {iteration} 回')

center1 = history[-1, :, :]

もはや初期値から殆ど動いていないが，目標のクラスター３に分類された３つの点が，相変わらず３のままであり，加えてクラスター２の中心がこれらから逃げているようにも見えるので，クラスター２の初期値をよりクラスター３に近いように誘導し，クラスター３の中心をより右側から開始する：

$$
m_2:\vctr{-1}{-1}\mapsto\vctr{0}{-2}\;\; m_3:\vctr{1}{-2}\mapsto\vctr{2}{-2}
$$


In [ ]:
#| echo: false
initial_points = np.array([[0,-2],[2,-2],[2.5,2]])
result = kmeans_2d(data1_2d, 3, initial_points.T, 0.1)
result.run_hard()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [2,0,1]
default = [0,1,2]
markerstyle = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markerstyle[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=1')

axs[0].legend()

# markerを使用したプロット
for marker in np.unique(markers1):
    axs[1].scatter(x1[markers1 == marker], y1[markers1 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

axs[1].set_title('Answer')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct, accr = accuracy(ans, normalize(pred))
iteration = history.shape[0]
print(f'正解数: {num_correct}     正解率: {accr*100:.1f} %     反復数: {iteration} 回')

center2 = history[-1, :, :]

こんなに誘導をしても，正しく分類してくれない．

実は，以上２つの初期値では，最終的に３つのクラスター中心は同じ値に収束している．よって，これ以上どのように初期値を変更しても，正答率は上がらないシナリオが考えられる．

以上の観察から，ハード $K$-平均法はある種の **局所解に収束する** ようなアルゴリズムであると考えられる．

## ソフト $K$-平均法

### アルゴリズムの説明 {#sec-soft-k-means}

ハード $K$-平均法[-@sec-hard-k-means]では，負担率
$$
r_{kn}\gets\delta_{k}(\argmax_{i\in[k]}d(m_i,x_n))
$$
は $0,1$ のいずれかの値しか取らなかった．この振る舞いを，
$$
\sigma(z;e)_i:=\frac{e^{z_i}}{\sum_{j=1}^Ke^{e_j}}\quad(i\in[K])
$$
で定まる [**ソフトマックス関数**](https://ja.wikipedia.org/wiki/%E3%82%BD%E3%83%95%E3%83%88%E3%83%9E%E3%83%83%E3%82%AF%E3%82%B9%E9%96%A2%E6%95%B0) $\sigma:\R^K\to(0,1)^K$ を用いて，「軟化」する．

ここでは，$\beta\ge0$ として，
$$
\sigma(z;e^{-\beta})_i=\frac{e^{-\beta z_i}}{\sum_{j=1}^Ke^{-\beta e_j}}
$$
の形で用い，$\argmax$ の代わりに
$$
\begin{align*}
    r_{kn}&\gets\sigma(d(-,x_n)^2\circ m;e^{-\beta})_k\\
    &=\frac{e^{-\beta d(m_k,x_n)^2}}{\sum_{j=1}^K e^{-\beta d(m_j,x_n)^2}}
\end{align*}
$$
とする．ただし，$d$ は $\R^2$ 上の Euclid 距離とした．

#### 硬度パラメータ

$\beta$ は **硬度 (stiffness)** または逆温度と呼ぶ．^[stiffness の用語は [@MacKay2003 p.289] から．実は各クラスターに Gauss モデルを置いた場合の分散 $\sigma^2$ に対して，$\beta=\frac{1}{2\sigma^2}$ の関係がある．[次稿](VI2.qmd#sec-EM-and-K-means) 参照．] $\sigma:=\beta^{-1/2}$ は距離の次元を持つ．

$\beta=0$ のときは温度が無限大の場合にあたり，常に負担率は一様になる．絶対零度に当たる $\beta\to\infty$ の極限が hard $K$-means アルゴリズムに相当する．

逆温度 $\beta$ を連続的に変化させることで，クラスタ数に分岐が起こる，ある種の相転移現象を見ることができる．^[[@MacKay2003 p.291]．]

#### 実装

実装は例えば hard $K$-means アルゴリズム[-@sec-hard-k-means]から，負担率計算の部分のみを変更すれば良い：

```{.python}
for i in range(N):
        distances = np.array([d(data[i], m[:,k]) for k in range(K)]) # <1>
        denominator_ = np.sum(np.exp(-beta * distances))  # <2>
        r[:,i] = np.exp(-beta * distances) / denominator_ # <3>
```
1. データ $x_i$ とクラスター中心 $(m_k)_{k=1}^K$ との距離を計算し，ベクトル $(d(x_n,m_k))_{k=1}^K$ を `distances` に格納している．
2. 負担率の計算
$$
r_{ik}=\frac{\exp(-\beta d(m_k,x_i))}{\sum_{j=1}^K\exp(-\beta d(m_j,x_i))}
$$
を２段階に分けて行なっており，分母を先に計算して変数 `denominator_` に格納している．
3. すでに計算してある分母 `denominator_` を用いてデータ $x_i$ の負担率 $(r_{ki})_{k=1}^K$ を計算し，$(K,N)$-行列 `r` の各列に格納している．

### 挙動の変化の観察

逆温度をはじめに $\beta=0.3$ としてみる．@fig-1 と全く同様な初期値
$$
m_1:=\vctr{4}{0},\quad m_2:=\vctr{1}{4},\quad m_3=\vctr{-1}{1},
$$
を与えてみると，次の通りの結果を得る：


In [ ]:
#| echo: false
#| label: fig-3
#| fig-cap: 左がソフト K-平均法（$\beta=1$），右がハード K-平均法によるクラスタリングの結果（図２の左と全く同じもの）．初期値は $(m_1,m_2,m_3)=\paren{\vctr{4}{0},\vctr{1}{4},\vctr{-1}{1}}$．赤丸で囲まれている点がクラスター中心 (CoC / Center of Cluster) の初期値で，その後の移動が図示されている．
initial_points = np.array([[-1,1],[1,4],[4,0]])
result = kmeans_2d(data1_2d, 3, initial_points.T, 0.3)
result.run_soft()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)

result2 = kmeans_2d(data1_2d, 3, initial_points.T, 1)
result2.run_hard()
pred2 = result2.fetch_cluster()
history2 = result2.fetch_history()

markers_pred2 = normalize(pred2)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [2,1,0]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.3')

axs[0].legend()

for marker in np.unique(markers_pred2):
    axs[1].scatter(x1[markers_pred2 == marker], y1[markers_pred2 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [2,1,0]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Hard K-means')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct1, accr1 = accuracy(ans, normalize(pred))
iteration = history.shape[0]
num_correct2, accr2 = accuracy(ans, normalize(pred2))
print(f'正解数: {num_correct1} vs. {num_correct2}     正解率: {accr1*100:.1f} % vs. {accr2*100:.1f} %     反復数: {history.shape[0]} 回 vs. {history2.shape[0]} 回')

クラスターの境界が変化しており，正解率は悪化している．さらに，反復数が９回であったところから，３倍に増えている（28回）．

また，右上の２つのクラスター中心の収束先は，微妙にずれているが **ほとんど一致している** 点も注目に値する．

::: {.callout-note icon="false" title="参考：最終的なクラスター中心の座標" collapse="true"}
```{.python}
centers = history[-1, :, :]
df = pd.DataFrame(centers, columns=['Cluster1', 'Cluster2', 'Cluster3'])
print(df)
```

In [ ]:
#| echo: false
center3 = history[-1, :, :]
df = pd.DataFrame(center3, columns=['Cluster1', 'Cluster2', 'Cluster3'])
df = df[['Cluster3', 'Cluster2', 'Cluster1']]
df.index = ['x', 'y']
df.columns = ['Cluster1', 'Cluster2', 'Cluster3']
print(df)

:::

@fig-2 で与えた初期値 $(m_1',m_2,m_3)$ も与えてみる．


In [ ]:
#| echo: false
#| label: fig-4
#| fig-cap: ソフト K-平均法（$\beta=1$）によるクラスタリングの結果，右がハード K-平均法によるクラスタリングの結果（図２の右と全く同じもの）．初期値は $(m_1',m_2,m_3)=\paren{\vctr{4}{-1},\vctr{1}{4},\vctr{-1}{1}}$．
initial_points = np.array([[-1,1],[1,4],[4,-1]])
result = kmeans_2d(data1_2d, 3, initial_points.T, 0.3)
result.run_soft()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)

result2 = kmeans_2d(data1_2d, 3, initial_points.T, 1)
result2.run_hard()
pred2 = result2.fetch_cluster()
history2 = result2.fetch_history()

markers_pred2 = normalize(pred2)

fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.3')

axs[0].legend()

for marker in np.unique(markers_pred2):
    axs[1].scatter(x1[markers_pred2 == marker], y1[markers_pred2 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title("Hard K-means")

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct1, accr1 = accuracy(ans, normalize_abnormal(pred))
iteration = history.shape[0]
num_correct2, accr2 = accuracy(ans, normalize(pred2))
print(f'正解数: {num_correct1} vs. {num_correct2}     正解率: {accr1*100:.1f} % vs. {accr2*100:.1f} %     反復数: {history.shape[0]} 回 vs. {history2.shape[0]} 回')

クラスター境界と正答率は変わらないが，反復数がやはり７回から大きく増えている．

結果はやはり @fig-3 とは大きく異なっており，ハード $K$-平均法で観察された初期値鋭敏性が，変わらず残っている．

加えてこの場合も @fig-3 のクラスター１と２と同様に，クラスター２と３の中心がほぼ一致している．

::: {.callout-note icon="false" title="参考：最終的なクラスター中心の座標" collapse="true"}

In [ ]:
#| echo: false
center4 = history[-1, :, :]
df = pd.DataFrame(center4, columns=['Cluster1', 'Cluster2', 'Cluster3'])
df = df[['Cluster2', 'Cluster1', 'Cluster3']]
df.index = ['x', 'y']
df.columns = ['Cluster1', 'Cluster2', 'Cluster3']
print(df)

:::

$\beta=0.3$ の場合のソフト $K$-平均法は，この例では **クラスター中心が融合する傾向にある** ようである．

一般に，$\beta$ が小さく，温度が大きいほど，エネルギーランドスケープに極小点が少なくなり，クラスターは同じ場所へ収束しやすくなると予想される．

### 高温になるほどクラスター数は減少する

初期値を直前で用いた
$$
m_1\gets\vctr{4}{-1},\quad m_2\gets\vctr{1}{4},\quad m_3\gets\vctr{-1}{1},
$$
で固定とし，さらに温度を上げて，逆温度を $\beta=0.1$ としてみる．


In [ ]:
#| echo: false
#| label: fig-5
#| fig-cap: ソフト K-平均法（左$\beta=0.1$，右$\beta=1$）によるクラスタリングの結果．初期値は $(m_1',m_2,m_3)=\paren{\vctr{4}{-1},\vctr{1}{4},\vctr{-1}{1}}$．
initial_points = np.array([[-1,1],[1,4],[4,-1]])
result = kmeans_2d(data1_2d, 3, initial_points.T, 0.1)
result.run_soft()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)

result2 = kmeans_2d(data1_2d, 3, initial_points.T, 0.3)
result2.run_soft()
pred2 = result2.fetch_cluster()
history2 = result2.fetch_history()

markers_pred2 = normalize(pred2)

fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.1')

axs[0].legend()

for marker in np.unique(markers_pred2):
    axs[1].scatter(x1[markers_pred2 == marker], y1[markers_pred2 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Soft K-means with beta=0.3')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct1, accr1 = accuracy(ans, normalize(pred))
iteration = history.shape[0]
num_correct2, accr2 = accuracy(ans, normalize_abnormal(pred2))
print(f'正解数: {num_correct1} vs. {num_correct2}     正解率: {accr1*100:.1f} % vs. {accr2*100:.1f} %     反復数: {history.shape[0]} 回 vs. {history2.shape[0]} 回')

反復数はさらに増加し，全てがほとんど同じクラスターに属する結果となってしまった．

::: {.callout-note icon="false" title="参考：最終的なクラスター中心の座標" collapse="true"}

In [ ]:
#| echo: false
center5 = history[-1, :, :]
df = pd.DataFrame(center5, columns=['Cluster1', 'Cluster2', 'Cluster3'])
df = df[['Cluster2', 'Cluster1', 'Cluster3']]
df.index = ['x', 'y']
df.columns = ['Cluster1', 'Cluster2', 'Cluster3']
print(df)

:::

温度が大変に高い状態では，全てが乱雑で，３つのクラスターが一様・公平に負担率を持つようになった．そのため，第一歩からほとんど全体の中心へと移動し，反復数が減る．

次に，温度を少し下げて，逆温度を $\beta=2$ としてみる．


In [ ]:
#| echo: false
#| label: fig-6
#| fig-cap: ソフト K-平均法（左$\beta=10$，右$\beta=1$）によるクラスタリングの結果．初期値は $(m_1',m_2,m_3)=\paren{\vctr{4}{-1},\vctr{1}{4},\vctr{-1}{1}}$．
initial_points = np.array([[-1,1],[1,4],[4,-1]])
result = kmeans_2d(data1_2d, 3, initial_points.T, 2)
result.run_soft()
pred = result.fetch_cluster()
history = result.fetch_history()
ans = data1.iloc[:, 0]

markers_pred = normalize(pred)

result2 = kmeans_2d(data1_2d, 3, initial_points.T, 0.3)
result2.run_soft()
pred2 = result2.fetch_cluster()
history2 = result2.fetch_history()

markers_pred2 = normalize(pred2)

fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred):
    axs[0].scatter(x1[markers_pred == marker], y1[markers_pred == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=2')

axs[0].legend()

for marker in np.unique(markers_pred2):
    axs[1].scatter(x1[markers_pred2 == marker], y1[markers_pred2 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Soft K-means with beta=0.3')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct1, accr1 = accuracy(ans, normalize(pred))
iteration = history.shape[0]
num_correct2, accr2 = accuracy(ans, normalize_abnormal(pred2))
print(f'正解数: {num_correct1} vs. {num_correct2}     正解率: {accr1*100:.1f} % vs. {accr2*100:.1f} %     反復数: {history.shape[0]} 回 vs. {history2.shape[0]} 回')

初めて soft $K$-means アルゴリズムを用いた場合で，３つのクラスター中心がはっきりと別れた．反復回数は，$\beta=0.3$ の場合と比べればやはり落ち着いている．

しかし，正解率は head $K$-means の場合（ @fig-2 など）と全く同じである．実は，最終的なクラスター中心も @fig-2 の最終的なクラスター中心とほとんど同じになっている．

::: {.callout-note icon="false" title="参考：最終的なクラスター中心の座標" collapse="true"}

**今回のソフト $K$-平均法の最終的なクラスター中心**


In [ ]:
#| echo: false
center6 = history[-1, :, :]
df1 = pd.DataFrame(center6, columns=['Cluster1', 'Cluster2', 'Cluster3'])
df1 = df1[['Cluster2', 'Cluster3', 'Cluster1']]
df1.index = ['x', 'y']
df1.columns = ['Cluster1', 'Cluster2', 'Cluster3']
print(df1)

@fig-2 のハード $K$-平均法の最終的なクラスター中心


In [ ]:
#| echo: false
df2 = pd.DataFrame(center_2, columns=['Cluster1', 'Cluster2', 'Cluster3'])
df2 = df2[['Cluster2', 'Cluster3', 'Cluster1']]
df2.index = ['x', 'y']
df2.columns = ['Cluster1', 'Cluster2', 'Cluster3']
print(df2)

:::

以上より，ソフト $K$-平均法は温度を上げるほどクラスター数が少なくなり，温度を下げるほどクラスター数は上がり，**十分に温度を下げるとハード $K$-平均法に挙動が似通う**．

### 最適な硬度の選択

$\beta=0.2$ ではクラスターが２つに縮退し，$\beta=1$ では hard $K$-means アルゴリズムの結果とほとんど変わらなくなる．その中間では次のように挙動が変わる：


In [ ]:
#| echo: false
#| label: fig-7
#| fig-cap: ソフト K-平均法によるクラスタリングの結果の比較（$\beta=0.2$ vs. $\beta=0.25$）．
initial_points = np.array([[-1,1],[1,4],[4,-1]])
result1 = kmeans_2d(data1_2d, 3, initial_points.T, 0.2)
result2 = kmeans_2d(data1_2d, 3, initial_points.T, 0.25)
result1.run_soft()
result2.run_soft()
pred1 = result1.fetch_cluster()
pred2 = result2.fetch_cluster()
history1 = result1.fetch_history()
history2 = result2.fetch_history()
ans = data1.iloc[:, 0]

markers_pred1 = normalize(pred1)
markers_pred2 = normalize(pred2)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred1):
    axs[0].scatter(x1[markers_pred1 == marker], y1[markers_pred1 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,2,0]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i in rearrange:  # 点の数だけ繰り返し
    x_coords = history1[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history1[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[2-i], label=f'CoC {3-i}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.2')

axs[0].legend()


# marker_predを使用したプロット
for marker in np.unique(markers_pred2):
    axs[1].scatter(x1[markers_pred2 == marker], y1[markers_pred2 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
for i in rearrange:  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[2-i], label=f'CoC {3-i}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Soft K-means with beta=0.25')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct1, accr1 = accuracy(ans, normalize_abnormal(pred1))
num_correct2, accr2 = accuracy(ans, normalize_abnormal(pred2))
print(f'正解数: {num_correct1} vs. {num_correct2}')
print(f'正解率: {accr1*100:.1f} % vs. {accr2*100:.1f} %')

In [ ]:
#| echo: false
#| label: fig-8
#| fig-cap: ソフト K-平均法によるクラスタリングの結果の比較（$\beta=0.3$ vs. $\beta=0.5$）．
initial_points = np.array([[-1,1],[1,4],[4,-1]])
result1 = kmeans_2d(data1_2d, 3, initial_points.T, 0.3)
result2 = kmeans_2d(data1_2d, 3, initial_points.T, 0.5)
result1.run_soft()
result2.run_soft()
pred1 = result1.fetch_cluster()
pred2 = result2.fetch_cluster()
history1 = result1.fetch_history()
history2 = result2.fetch_history()
ans = data1.iloc[:, 0]

markers_pred1 = normalize(pred1)
markers_pred2 = normalize(pred2)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred1):
    axs[0].scatter(x1[markers_pred1 == marker], y1[markers_pred1 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
default = [0,1,2]
markers = ['o', '^', 's']  # マーカーの形（丸、三角、正方形）
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history1[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history1[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.3')

axs[0].legend()


# marker_predを使用したプロット
for marker in np.unique(markers_pred2):
    axs[1].scatter(x1[markers_pred2 == marker], y1[markers_pred2 == marker], label=f'Cluster {marker}', marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,2]
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Soft K-means with beta=0.5')

axs[1].legend()

plt.tight_layout()
plt.show()

num_correct1, accr1 = accuracy(ans, normalize_abnormal(pred1))
num_correct2, accr2 = accuracy(ans, normalize(pred2))
print(f'正解数: {num_correct1} vs. {num_correct2}')
print(f'正解率: {accr1*100:.1f} % vs. {accr2*100:.1f} %')

やはり，温度が高い場合はクラスター中心が合流・融合してしまいやすいが，冷却することでクラスター数は大きい状態で安定する，と言えるだろう．

## 本番データセットでの実験

今まで使っていたデータ[-@sec-data]はクラスターのオーバーラップはなかったため，いわば優しいデータであった．ここからはよりデータ生成過程が複雑なデータを用いて，ソフト $K$-平均法の挙動を観察する．

### データの概観 {#sec-data2}

今度は，次の４クラスのデータを用いる．


In [ ]:
#| echo: false

data2_unsorted = pd.read_csv('mixture2.dat', delimiter="\t" , header=None)
data2 = data2_unsorted.sort_values(by=0, ascending=True)
data2_2d = data2.iloc[:, 1:3].to_numpy()

markers2 = data2.iloc[:, 0]
x2 = data2.iloc[:, 1]
y2 = data2.iloc[:, 2]

plt.figure(figsize=(3.5, 3))
for marker in np.unique(markers2):
    plt.scatter(x2[markers2 == marker], y2[markers2 == marker], label=f'Cluster {marker}', marker=f'${marker}$') 

plt.title('Scatter Plot of the Data')
plt.show()

実は，これは４つの Gauss 分布から生成されたデータである．

### 最適な温度の選択


In [ ]:
#| echo: false
initial_points = np.array([[1,6],[6,-1],[1,-8],[-4,-1]])
result1 = kmeans_2d(data2_2d, 4, initial_points.T, 0.1)
result2 = kmeans_2d(data2_2d, 4, initial_points.T, 0.15)
result1.run_soft()
result2.run_soft()
pred1 = result1.fetch_cluster()
pred2 = result2.fetch_cluster()
history1 = result1.fetch_history()
history2 = result2.fetch_history()
ans = data2.iloc[:, 0]

markers_pred1 = normalize(pred1)
markers_pred2 = normalize(pred2)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred1):
    axs[0].scatter(x2[markers_pred1 == marker], y2[markers_pred1 == marker], marker=f'${marker}$', alpha=0.3),  # markerパラメータによって形状を指定

rearrange = [0,1,2,3]
default = [0,1,2,3]
labels = [2,1,3,2]
markers = ['o', '^', 's', '*']
colors = ['orange', 'blue', 'green', 'orange']
for i,j,k in zip(rearrange, default, labels):  # 点の数だけ繰り返し
    x_coords = history1[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history1[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, color=colors[j], linestyle='-', marker=markers[j], label=f'CoC {k}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.1')
axs[0].legend()


# marker_predを使用したプロット
for marker in np.unique(markers_pred2):
    axs[1].scatter(x2[markers_pred2 == marker], y2[markers_pred2 == marker], marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,3,2]
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history1[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history1[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Soft K-means with beta=0.15')
axs[1].legend()

plt.tight_layout()
plt.show()

num_correct1, accr1 = accuracy4(ans, normalize(pred1))
num_correct2, accr2 = accuracy4(ans, normalize(pred2))
iteration1 = history1.shape[0]
iteration2 = history2.shape[0]

In [ ]:
#| echo: false
result1 = kmeans_2d(data2_2d, 4, initial_points.T, 0.2)
result2 = kmeans_2d(data2_2d, 4, initial_points.T, 0.3)
result1.run_soft()
result2.run_soft()
pred1 = result1.fetch_cluster()
pred2 = result2.fetch_cluster()
history1 = result1.fetch_history()
history2 = result2.fetch_history()
ans = data2.iloc[:, 0]

markers_pred1 = normalize(pred1)
markers_pred2 = normalize(pred2)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred1):
    axs[0].scatter(x2[markers_pred1 == marker], y2[markers_pred1 == marker], marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,3,2]
default = [0,1,2,3]
markers = ['o', '^', 's', '*']
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history1[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history1[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.2')
axs[0].legend()


# marker_predを使用したプロット
for marker in np.unique(markers_pred2):
    axs[1].scatter(x2[markers_pred2 == marker], y2[markers_pred2 == marker], marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Soft K-means with beta=0.3')
axs[1].legend()

plt.tight_layout()
plt.show()

num_correct3, accr3 = accuracy4(ans, normalize(pred1))
num_correct4, accr4 = accuracy4(ans, normalize(pred2))
iteration3 = history1.shape[0]
iteration4 = history2.shape[0]

In [ ]:
#| echo: false
result1 = kmeans_2d(data2_2d, 4, initial_points.T, 0.5)
result2 = kmeans_2d(data2_2d, 4, initial_points.T, 1)
result1.run_soft()
result2.run_hard()
pred1 = result1.fetch_cluster()
pred2 = result2.fetch_cluster()
history1 = result1.fetch_history()
history2 = result2.fetch_history()
ans = data2.iloc[:, 0]

markers_pred1 = normalize(pred1)
markers_pred2 = normalize(pred2)
fig, axs = plt.subplots(1, 2, figsize=(7, 4))  # 1行2列のサブプロット

# marker_predを使用したプロット
for marker in np.unique(markers_pred1):
    axs[0].scatter(x2[markers_pred1 == marker], y2[markers_pred1 == marker], marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

rearrange = [1,0,3,2]
default = [0,1,2,3]
markers = ['o', '^', 's', '*']
for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history1[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history1[:, 1, i]  # i番目の点の全時点でのy座標
    axs[0].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[0].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[0].set_title('Soft K-means with beta=0.5')
axs[0].legend()


# marker_predを使用したプロット
for marker in np.unique(markers_pred2):
    axs[1].scatter(x2[markers_pred2 == marker], y2[markers_pred2 == marker], marker=f'${marker}$', alpha=0.3)  # markerパラメータによって形状を指定

for i,j in zip(rearrange, default):  # 点の数だけ繰り返し
    x_coords = history2[:, 0, i]  # i番目の点の全時点でのx座標
    y_coords = history2[:, 1, i]  # i番目の点の全時点でのy座標
    axs[1].plot(x_coords, y_coords, linestyle='-', marker=markers[j], label=f'CoC {j+1}', zorder=1)

axs[1].scatter(initial_points[:, 0], initial_points[:, 1], facecolors='none', edgecolors='red', s=100, zorder=2)

axs[1].set_title('Hard K-means')
axs[1].legend()

plt.tight_layout()
plt.show()

num_correct5, accr5 = accuracy4(ans, normalize(pred1))
num_correct6, accr6 = accuracy4(ans, normalize(pred2))
iteration5 = history1.shape[0]
iteration6 = history2.shape[0]

In [ ]:
#| echo: false
print(f'正解数: {num_correct1} vs. {num_correct2}     正解率: {accr1*100:.1f} % vs. {accr2*100:.1f} %     反復数: {iteration1} 回 vs. {iteration2} 回')
print(f'正解数: {num_correct3} vs. {num_correct4}     正解率: {accr3*100:.1f} % vs. {accr4*100:.1f} %     反復数: {iteration3} 回 vs. {iteration4} 回')
print(f'正解数: {num_correct5} vs. {num_correct6}     正解率: {accr5*100:.1f} % vs. {accr6*100:.1f} %     反復数: {iteration5} 回 vs. {iteration6} 回')

## 実験結果まとめ {#sec-result}

::: {.callout-note icon="false" title="結論"}
* データ [-@sec-data] に対して，（初期値 $(m'_1,m_2,m_3)$ で）ソフト $K$-平均法を適用すると，
  * $\beta\ge2$ の場合で結果はハード $K$-平均法と変わらなくなる．
  * $\beta=1$ の場合で結果はクラスターがほとんど２つになり，$\beta\le0.5$ では計算機上では実際に２つになってしまう．
  * 正答率は $1\le\beta\le1.1$ で最大であった．
  * $\beta$ を大きくするほど，反復回数は減少していった．
* データ [-@sec-data2] に対しても，以上の４点について同様の傾向が確認できた．
:::

こうしてソフト $K$-平均法とハード $K$-平均法の性質は分かった．主に

1. 初期値依存性
2. クラスタ数 $K$ の選択法

の問題が未解決であり，恣意性が残る．

これを，ベイズ混合モデリングにより解決する方法を[次稿](VI2.qmd)で紹介する．